## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.97,72,97,11.07,light rain
1,1,Ciudad Guayana,VE,8.3533,-62.6528,83.52,67,41,7.00,scattered clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,51.91,43,0,10.36,clear sky
3,3,Qaanaaq,GL,77.4840,-69.3632,-7.37,61,96,8.88,overcast clouds
4,4,Kapaa,US,22.0752,-159.3190,66.18,88,0,6.91,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.97,72,97,11.07,light rain
1,1,Ciudad Guayana,VE,8.3533,-62.6528,83.52,67,41,7.00,scattered clouds
9,9,Bambous Virieux,MU,-20.3428,57.7575,80.22,94,75,17.27,light rain
10,10,Praya,CV,14.9215,-23.5087,84.74,32,0,18.41,clear sky
14,14,Butaritari,KI,3.0707,172.7902,82.02,75,11,17.00,light rain


In [10]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                216
City                   216
Country                214
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.97,light rain,-23.1203,-134.9692,
1,Ciudad Guayana,VE,83.52,scattered clouds,8.3533,-62.6528,
9,Bambous Virieux,MU,80.22,light rain,-20.3428,57.7575,
10,Praya,CV,84.74,clear sky,14.9215,-23.5087,
14,Butaritari,KI,82.02,light rain,3.0707,172.7902,
21,Calderitas,MX,82.51,scattered clouds,18.5500,-88.2500,
22,Souillac,MU,80.37,light rain,-20.5167,57.5167,
24,Fougamou,GA,82.02,light rain,-1.2154,10.5838,
25,Rio Verde De Mato Grosso,BR,86.23,broken clouds,-18.9181,-54.8442,
28,Hermanus,ZA,78.91,clear sky,-34.4187,19.2345,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
        lat = row["Lat"]
        lng = row["Lng"]
        
        # Add the lat and lng to location key for the params dictionary
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
   
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.count()
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [23]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.97,light rain,-23.1203,-134.9692,People ThankYou
1,Ciudad Guayana,VE,83.52,scattered clouds,8.3533,-62.6528,Hotel Las Americas
9,Bambous Virieux,MU,80.22,light rain,-20.3428,57.7575,Casa Tia Villa
10,Praya,CV,84.74,clear sky,14.9215,-23.5087,LT Aparthotel
14,Butaritari,KI,82.02,light rain,3.0707,172.7902,Isles Sunset Lodge
21,Calderitas,MX,82.51,scattered clouds,18.5500,-88.2500,Hotel Hacienda Campestre
22,Souillac,MU,80.37,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
24,Fougamou,GA,82.02,light rain,-1.2154,10.5838,Résidence d'Hôtes Bapedy
25,Rio Verde De Mato Grosso,BR,86.23,broken clouds,-18.9181,-54.8442,Quedas Palace Hotel
28,Hermanus,ZA,78.91,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel


In [24]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))